#Dataset

In [2]:
import cv2 # OpenCV
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import os
import tensorflow_datasets as tfds
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
path = "FULL"
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   validation_split=0.5)
training_set = train_datagen.flow_from_directory(path+"/train",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,
                                                class_mode='categorical',
                                                subset='training')

validation_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True)
validation_set = train_datagen.flow_from_directory(path+"/test",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,
                                                class_mode='categorical',
                                                subset='training')

Found 15923 images belonging to 7 classes.
Found 3976 images belonging to 7 classes.


#VGG16

In [5]:
VGG16=tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None
)
VGG16.trainable = True
print(len(VGG16.layers))

i = 0
for layer in VGG16.layers:
  if i > 11:
    layer.trainable = False
  i+=1

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),  # Rotate up to 20 degrees
    tf.keras.layers.RandomTranslation(0.1, 0.15),  # Translate 10% width, 15% height
    tf.keras.layers.RandomBrightness(factor=[0.2, 1.0]),  # Change brightness between 0.2 and 1
    tf.keras.layers.RandomZoom(height_factor=(-0.1, 0.2), width_factor=(-0.1, 0.2)),  # Zoom out up to 10%, zoom in up to 20%
    tf.keras.layers.RandomFlip("horizontal"),  # Flip horizontally
])

input = tf.keras.layers.Input(shape=(224, 224, 3))
x = data_augmentation(input)
x = VGG16(input, training=False)
x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(7, activation = 'softmax')(x)
VGG16 = tf.keras.models.Model(inputs = input, outputs = x, name = "VGG16")

VGG16.summary()

19
Model: "VGG16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 average_pooling2d (AverageP  (None, 4, 4, 512)        0         
 ooling2D)                                                       
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 7)                 57351     
                                                                 
Total params: 14,772,039
Trainable params: 2,972,999
Non-trainable params: 11,799,040
______________________________________

In [7]:
steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
batch_size = 40
history = VGG16.fit(x=training_set,
                    epochs=40,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )
VGG16.save('VGG16Augmented_Model_Reverse.keras')

Epoch 1/40
31/31 [==============================] - 880s 28s/step - loss: 1.8925 - accuracy: 0.2061
Epoch 2/40
31/31 [==============================] - 837s 27s/step - loss: 1.8717 - accuracy: 0.2314
Epoch 3/40
31/31 [==============================] - 772s 25s/step - loss: 1.8623 - accuracy: 0.2420
Epoch 4/40
31/31 [==============================] - 822s 27s/step - loss: 1.8442 - accuracy: 0.2152
Epoch 5/40
31/31 [==============================] - 802s 26s/step - loss: 1.8506 - accuracy: 0.2223
Epoch 6/40
31/31 [==============================] - 855s 28s/step - loss: 1.8500 - accuracy: 0.2177
Epoch 7/40
31/31 [==============================] - 788s 25s/step - loss: 1.8358 - accuracy: 0.2440
Epoch 8/40
31/31 [==============================] - 777s 25s/step - loss: 1.8071 - accuracy: 0.2566
Epoch 9/40
31/31 [==============================] - 785s 25s/step - loss: 1.7819 - accuracy: 0.2903
Epoch 10/40
31/31 [==============================] - 737s 24s/step - loss: 1.7364 - accuracy: 0.3176

In [3]:
VGG16 = tf.keras.models.load_model('VGG16Augmented_Model_Reverse.keras')

steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
history = VGG16.fit(x=training_set,
                    epochs=60,
                    steps_per_epoch=steps_per_epoch
                    )
VGG16.save('VGG16Augmented_Model_Reverse.keras')

Epoch 1/60
31/31 [==============================] - 784s 25s/step - loss: 1.3775 - accuracy: 0.4798
Epoch 2/60
31/31 [==============================] - 724s 23s/step - loss: 1.3629 - accuracy: 0.4934
Epoch 3/60
31/31 [==============================] - 731s 24s/step - loss: 1.3674 - accuracy: 0.4768
Epoch 4/60
31/31 [==============================] - 730s 24s/step - loss: 1.3520 - accuracy: 0.4849
Epoch 5/60
31/31 [==============================] - 727s 23s/step - loss: 1.3104 - accuracy: 0.5010
Epoch 6/60
31/31 [==============================] - 719s 23s/step - loss: 1.2946 - accuracy: 0.5091
Epoch 7/60
31/31 [==============================] - 733s 24s/step - loss: 1.3080 - accuracy: 0.5055
Epoch 8/60
31/31 [==============================] - 748s 24s/step - loss: 1.2956 - accuracy: 0.5040
Epoch 9/60
31/31 [==============================] - 728s 23s/step - loss: 1.2938 - accuracy: 0.5190
Epoch 10/60
31/31 [==============================] - 718s 23s/step - loss: 1.3217 - accuracy: 0.4894

In [4]:
VGG16 = tf.keras.models.load_model('VGG16Augmented_Model_Reverse.keras')

steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
history = VGG16.fit(x=training_set,
                    epochs=30,
                    steps_per_epoch=steps_per_epoch
                    )
VGG16.save('VGG16Augmented_Model_Reverse.keras')

Epoch 1/30
31/31 [==============================] - 720s 23s/step - loss: 1.0858 - accuracy: 0.6053
Epoch 2/30
31/31 [==============================] - 718s 23s/step - loss: 1.0655 - accuracy: 0.6033
Epoch 3/30
31/31 [==============================] - 731s 23s/step - loss: 1.0563 - accuracy: 0.6109
Epoch 4/30
31/31 [==============================] - 731s 24s/step - loss: 1.0183 - accuracy: 0.6220
Epoch 5/30
31/31 [==============================] - 734s 24s/step - loss: 1.0682 - accuracy: 0.6008
Epoch 6/30
31/31 [==============================] - 734s 24s/step - loss: 1.0385 - accuracy: 0.6064
Epoch 7/30
31/31 [==============================] - 716s 23s/step - loss: 1.0247 - accuracy: 0.6230
Epoch 8/30
31/31 [==============================] - 705s 23s/step - loss: 1.0459 - accuracy: 0.5998
Epoch 9/30
31/31 [==============================] - 711s 23s/step - loss: 1.0220 - accuracy: 0.6220
Epoch 10/30
31/31 [==============================] - 740s 24s/step - loss: 1.0307 - accuracy: 0.6079

In [2]:
VGG16 = tf.keras.models.load_model('VGG16Augmented_Model_Reverse.keras')

steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
history = VGG16.fit(x=training_set,
                    epochs=70,
                    steps_per_epoch=steps_per_epoch
                    )
VGG16.save('VGG16Augmented_Model_Reverse.keras')

Epoch 1/70
31/31 [==============================] - 768s 24s/step - loss: 0.9445 - accuracy: 0.6447
Epoch 2/70
31/31 [==============================] - 711s 23s/step - loss: 0.9339 - accuracy: 0.6552
Epoch 3/70
31/31 [==============================] - 738s 24s/step - loss: 0.9095 - accuracy: 0.6709
Epoch 4/70
31/31 [==============================] - 711s 23s/step - loss: 0.8832 - accuracy: 0.6764
Epoch 5/70
31/31 [==============================] - 721s 23s/step - loss: 0.9231 - accuracy: 0.6643
Epoch 6/70
31/31 [==============================] - 740s 24s/step - loss: 0.8922 - accuracy: 0.6643
Epoch 7/70
31/31 [==============================] - 728s 23s/step - loss: 0.8828 - accuracy: 0.6799
Epoch 8/70
31/31 [==============================] - 704s 23s/step - loss: 0.8690 - accuracy: 0.6774
Epoch 9/70
31/31 [==============================] - 720s 23s/step - loss: 0.9041 - accuracy: 0.6651
Epoch 10/70
31/31 [==============================] - 723s 23s/step - loss: 0.9051 - accuracy: 0.6754

In [3]:
VGG16 = tf.keras.models.load_model('VGG16Augmented_Model_Reverse.keras')

steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
history = VGG16.fit(x=training_set,
                    epochs=10,
                    steps_per_epoch=steps_per_epoch
                    )
VGG16.save('VGG16Augmented_Model_Reverse.keras')

Epoch 1/10
31/31 [==============================] - 807s 26s/step - loss: 0.6145 - accuracy: 0.7792
Epoch 2/10
31/31 [==============================] - 711s 23s/step - loss: 0.6130 - accuracy: 0.7908
Epoch 3/10
31/31 [==============================] - 819s 26s/step - loss: 0.5631 - accuracy: 0.7933
Epoch 4/10
31/31 [==============================] - 718s 23s/step - loss: 0.5344 - accuracy: 0.8070
Epoch 5/10
31/31 [==============================] - 707s 23s/step - loss: 0.5593 - accuracy: 0.7944
Epoch 6/10
31/31 [==============================] - 703s 23s/step - loss: 0.5575 - accuracy: 0.8004
Epoch 7/10
31/31 [==============================] - 765s 25s/step - loss: 0.5080 - accuracy: 0.8221
Epoch 8/10
31/31 [==============================] - 702s 23s/step - loss: 0.5398 - accuracy: 0.8090
Epoch 9/10
31/31 [==============================] - 735s 24s/step - loss: 0.5310 - accuracy: 0.8039
Epoch 10/10
31/31 [==============================] - 746s 24s/step - loss: 0.5491 - accuracy: 0.7965

In [4]:
VGG16 = tf.keras.models.load_model('VGG16Augmented_Model_Reverse.keras')

steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
history = VGG16.fit(x=training_set,
                    epochs=10,
                    steps_per_epoch=steps_per_epoch
                    )
VGG16.save('VGG16Augmented_Model_Reverse.keras')

Epoch 1/10
31/31 [==============================] - 752s 24s/step - loss: 0.6211 - accuracy: 0.7676
Epoch 2/10
31/31 [==============================] - 734s 24s/step - loss: 0.5122 - accuracy: 0.8185
Epoch 3/10
31/31 [==============================] - 736s 24s/step - loss: 0.4824 - accuracy: 0.8256
Epoch 4/10
31/31 [==============================] - 728s 23s/step - loss: 0.5070 - accuracy: 0.8255
Epoch 5/10
31/31 [==============================] - 735s 24s/step - loss: 0.5051 - accuracy: 0.8216
Epoch 6/10
31/31 [==============================] - 726s 23s/step - loss: 0.5164 - accuracy: 0.8120
Epoch 7/10
31/31 [==============================] - 735s 24s/step - loss: 0.5344 - accuracy: 0.8070
Epoch 8/10
31/31 [==============================] - 787s 25s/step - loss: 0.5327 - accuracy: 0.8130
Epoch 9/10
31/31 [==============================] - 783s 25s/step - loss: 0.4880 - accuracy: 0.8266
Epoch 10/10
31/31 [==============================] - 793s 26s/step - loss: 0.4949 - accuracy: 0.8226

In [2]:
VGG16 = tf.keras.models.load_model('VGG16Augmented_Model_Reverse.keras')

steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
history = VGG16.fit(x=training_set,
                    epochs=35,
                    steps_per_epoch=steps_per_epoch
                    )
VGG16.save('VGG16Augmented_Model_Reverse.keras')

Epoch 1/35
31/31 [==============================] - 876s 28s/step - loss: 0.4862 - accuracy: 0.8295
Epoch 2/35
31/31 [==============================] - 850s 27s/step - loss: 0.5275 - accuracy: 0.8049
Epoch 3/35
31/31 [==============================] - 843s 27s/step - loss: 0.5249 - accuracy: 0.8085
Epoch 4/35
31/31 [==============================] - 829s 27s/step - loss: 0.4682 - accuracy: 0.8236
Epoch 5/35
31/31 [==============================] - 828s 27s/step - loss: 0.4624 - accuracy: 0.8352
Epoch 6/35
31/31 [==============================] - 826s 27s/step - loss: 0.4877 - accuracy: 0.8291
Epoch 7/35
31/31 [==============================] - 839s 27s/step - loss: 0.4636 - accuracy: 0.8337
Epoch 8/35
31/31 [==============================] - 841s 27s/step - loss: 0.4587 - accuracy: 0.8337
Epoch 9/35
31/31 [==============================] - 850s 27s/step - loss: 0.4703 - accuracy: 0.8342
Epoch 10/35
31/31 [==============================] - 845s 27s/step - loss: 0.4213 - accuracy: 0.8558

In [3]:
VGG16 = tf.keras.models.load_model('VGG16Augmented_Model_Reverse.keras')

steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
history = VGG16.fit(x=training_set,
                    epochs=40,
                    steps_per_epoch=steps_per_epoch
                    )
VGG16.save('VGG16Augmented_Model_Reverse.keras')

Epoch 1/40
31/31 [==============================] - 846s 27s/step - loss: 0.3918 - accuracy: 0.8669
Epoch 2/40
31/31 [==============================] - 722s 23s/step - loss: 0.3739 - accuracy: 0.8674
Epoch 3/40
31/31 [==============================] - 753s 24s/step - loss: 0.3331 - accuracy: 0.8826
Epoch 4/40
31/31 [==============================] - 715s 23s/step - loss: 0.3191 - accuracy: 0.8894
Epoch 5/40
31/31 [==============================] - 728s 23s/step - loss: 0.3364 - accuracy: 0.8851
Epoch 6/40
31/31 [==============================] - 742s 24s/step - loss: 0.3281 - accuracy: 0.8790
Epoch 7/40
31/31 [==============================] - 726s 24s/step - loss: 0.3209 - accuracy: 0.8919
Epoch 8/40
31/31 [==============================] - 755s 24s/step - loss: 0.3012 - accuracy: 0.8896
Epoch 9/40
31/31 [==============================] - 774s 25s/step - loss: 0.3114 - accuracy: 0.8942
Epoch 10/40
31/31 [==============================] - 840s 27s/step - loss: 0.3309 - accuracy: 0.8886

In [3]:
VGG16 = tf.keras.models.load_model('VGG16Augmented_Model_Reverse.keras')

steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
history = VGG16.fit(x=training_set,
                    epochs=35,
                    steps_per_epoch=steps_per_epoch
                    )
VGG16.save('VGG16Augmented_Model_Reverse.keras')

Epoch 1/35
31/31 [==============================] - 797s 26s/step - loss: 0.3012 - accuracy: 0.9061
Epoch 2/35
31/31 [==============================] - 779s 25s/step - loss: 0.2468 - accuracy: 0.9057
Epoch 3/35
31/31 [==============================] - 739s 24s/step - loss: 0.2505 - accuracy: 0.9138
Epoch 4/35
31/31 [==============================] - 794s 26s/step - loss: 0.1985 - accuracy: 0.9254
Epoch 5/35
31/31 [==============================] - 784s 25s/step - loss: 0.1942 - accuracy: 0.9361
Epoch 6/35
31/31 [==============================] - 797s 26s/step - loss: 0.2390 - accuracy: 0.9158
Epoch 7/35
31/31 [==============================] - 780s 25s/step - loss: 0.2793 - accuracy: 0.9027
Epoch 8/35
31/31 [==============================] - 788s 25s/step - loss: 0.2808 - accuracy: 0.9068
Epoch 9/35
31/31 [==============================] - 726s 23s/step - loss: 0.2412 - accuracy: 0.9103
Epoch 10/35
31/31 [==============================] - 764s 24s/step - loss: 0.2272 - accuracy: 0.9244